# LFM梯度下降算法实现

### 0. 引入依赖

In [8]:
import numpy as np
import pandas as pd

### 1. 数据准备

In [9]:
# 评分矩阵R
R = np.array([[4,0,2,0,1],
             [0,2,3,0,0],
             [1,0,2,4,0],
             [5,0,0,3,1],
             [0,0,1,5,1],
             [0,3,2,4,1],])
len(R[0])

5

### 2. 算法实现

In [10]:
"""
@输入参数：
R：M*N 的评分矩阵
K：隐特征向量维度
max_iter: 最大迭代次数
alpha：步长
lamda：正则化系数

@输出：
分解之后的 P，Q
P：初始化用户特征矩阵M*K
Q：初始化物品特征矩阵N*K
"""

# 给定超参数

K = 5
max_iter = 5000
alpha = 0.0002
lamda = 0.004

# 核心算法
def LFM_grad_desc( R, K=2, max_iter=1000, alpha=0.0001, lamda=0.002 ):
    # 基本维度参数定义
    M = len(R)
    N = len(R[0])
    
    # P,Q初始值，随机生成
    P = np.random.rand(M, K)
    Q = np.random.rand(N, K)
    Q = Q.T
    
    # 开始迭代
    for step in range(max_iter):
        # 对所有的用户u、物品i做遍历，对应的特征向量Pu、Qi梯度下降
        for u in range(M):
            for i in range(N):
                # 对于每一个大于0的评分，求出预测评分误差
                if R[u][i] > 0:
                    eui = np.dot( P[u,:], Q[:,i] ) - R[u][i]
                    
                    # 代入公式，按照梯度下降算法更新当前的Pu、Qi
                    for k in range(K):
                        P[u][k] = P[u][k] - alpha * ( 2 * eui * Q[k][i] + 2 * lamda * P[u][k] )
                        Q[k][i] = Q[k][i] - alpha * ( 2 * eui * P[u][k] + 2 * lamda * Q[k][i] )
        
        # u、i遍历完成，所有特征向量更新完成，可以得到P、Q，可以计算预测评分矩阵
        predR = np.dot( P, Q )
        
        # 计算当前损失函数
        cost = 0
        for u in range(M):
            for i in range(N):
                if R[u][i] > 0:
                    cost += ( np.dot( P[u,:], Q[:,i] ) - R[u][i] ) ** 2
                    # 加上正则化项
                    for k in range(K):
                        cost += lamda * ( P[u][k] ** 2 + Q[k][i] ** 2 )
        if cost < 0.0001:
            break
        
    return P, Q.T, cost

### 3. 测试

In [15]:
P, Q, cost = LFM_grad_desc(R, K, max_iter, alpha, lamda)

print('用户特征集：',P)
print('物品特征集：',Q)
print('误差值：',cost)

predR = P.dot(Q.T)

print('真实评分矩阵',R)
print('预测评分矩阵',predR)

用户特征集： [[ 0.65986037  0.6615279   1.12682422  0.73672687  0.49158554]
 [ 0.02105066  0.82504593  0.99252823  0.56393677  1.0740744 ]
 [ 0.90071919 -0.28492663  0.12029045  0.97553115  1.23721039]
 [ 0.57771895  1.57336358  0.89477818 -0.06814421  0.45337703]
 [ 0.96611633  0.50958414  1.23119177  1.12562581 -0.18798259]
 [ 0.97758696  0.86784976  0.70978234  0.49052708  0.65357826]]
物品特征集： [[ 0.70656266  1.94812106  1.60276124  0.41986848  0.26005317]
 [ 1.21000809  0.53113646  0.47754313  0.69308393  0.75260978]
 [-0.08679018  0.76846425  0.42619275  0.43419138  1.47734029]
 [ 1.17452725  0.45790149  1.44896174  1.76769296  0.93358308]
 [ 0.63120012  0.01134753  0.44812118 -0.11597626  0.31757394]]
误差值： 0.6381403475794173
真实评分矩阵 [[4 0 2 0 1]
 [0 2 3 0 0]
 [1 0 2 4 0]
 [5 0 0 3 1]
 [0 0 1 5 1]
 [0 3 2 4 1]]
预测评分矩阵 [[3.99816609 2.56849079 1.97745503 4.47190663 0.9996364 ]
 [3.72904445 2.1368729  2.88682952 3.84025451 0.74311711]
 [1.00547513 2.60324808 2.00548539 3.98122541 0.89897289]
